In [1]:
%run ds2.py
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageOps


In [7]:
def genspacing(savepath='', text = 'text', spacing = 0, index=1, mirror=False, invert=False, fontname='Arial', W = 500, H = 500, size=24, upper=0, show=None):
    if upper:
        text = text.upper()
    if invert:
        text = text[::-1]
    img = Image.new("RGB", (W,H), color = (255, 255, 255))
    fnt = ImageFont.truetype(fontname+'.ttf', size)
    draw = ImageDraw.Draw(img)
    
    if spacing == 0:
        w, h = fnt.getsize(text)
        draw.text(( (W-w)/2,  (H-h)/2), text, font=fnt, fill='black')

    else:
        
        total_text_width, h = draw.textsize(text, font=fnt)
        width_difference = total_text_width*spacing
        gap_width = int(width_difference / (len(text) - 1))
        xpos = (W - total_text_width*(1+spacing))/2


        for letter in text:
            draw.text((xpos,(H-h)/2),letter, (0,0,0), font=fnt)
            letter_width, letter_height = draw.textsize(letter, font=fnt)
            xpos += letter_width + gap_width

    
    img.save(savepath+text+str(index).zfill(2)+'.jpg')

    


In [ ]:
def genrotation(savepath='', text = 'text', rotation = 0, index=1, mirror=False, invert=False, fontname='Arial', W = 500, H = 500, size=24, xshift=0, yshift=0, upper=0, show=None):
    if upper:
        text = text.upper()
    if invert:
        text = text[::-1]
        
    img = Image.new("RGB", (W,H), color = (255, 255, 255))
    fnt = ImageFont.truetype(fontname+'.ttf', size)
    w, h = fnt.getsize(text)

    txt=Image.new('L', (W,H))
    draw = ImageDraw.Draw(txt)
    draw.text((xshift + (W-w)/2, yshift + (H-h)/2), text, font=fnt, fill= 255)
    
    xx = txt.rotate(rotation, expand=1)
    img.paste(ImageOps.colorize(xx,  (0,0,0), (0,0,0)), mask =xx)

        
    img.save(savepath+text+str(index)+'.jpg')



In [2]:
def gen2(savepath='', text = 'text', index=1, mirror=False, invert=False, fontname='Arial', W = 500, H = 500, size=24, xshift=0, yshift=0, upper=0, show=None):
    if upper:
        text = text.upper()
    if invert:
        text = text[::-1]
    img = Image.new("RGB", (W,H), color = (255, 255, 255))
    #fnt = ImageFont.truetype('/Library/Fonts/'+fontname+'.ttf', size) #size in pixels
    fnt = ImageFont.truetype(fontname+'.ttf', size)
    draw = ImageDraw.Draw(img)
    w, h = fnt.getsize(text)
    draw.text((xshift + (W-w)/2, yshift + (H-h)/2), text, font=fnt, fill='black')

    if mirror:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)
    if savepath != '':
        img.save(savepath+text+str(index)+'.jpg')
    if show:
        img.save('plots/'+text+str(index)+'.jpg')
        img.show()
    if savepath == '':
        print('I was here')
        img.show()
        return img


In [ ]:
# generating stimulus with random variations
def CreateMiniWordSet(path_out='wordsets_200cat_100ex/',num_train=100, num_val=0):
    #define words, sizes, fonts
#     words = words4yo()
    #random.shuffle(words)
    #wordlist =  ['dimanche', 'lundi', 'mots', 'samedi', 'semaine']
    words = np.load('words4yo.npy')
    wordlist = words[:200]
    sizes = [40, 50, 60, 70, 80]
    fonts = ['arial','tahoma']
    xshift = [-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50]
    yshift = [-30, -15, 0, 15, 30]
    
    #create train and val folders 
    for m in ['train', 'val']:
        for f in wordlist:
            target_path = path_out+m+'/'+f
            os.makedirs(target_path)
    
    #for each word, create num_train + num_val exemplars, then split randomly into train and val.
    for w in tqdm(wordlist):
        gc.collect()
        print (w,)
        for n in range(num_train + num_val):
            if n < num_train:
                path = path_out+'train/'+w+'/'
            else:
                path = path_out+'val/'+w+'/'
            
            f = random.choice(fonts)
            s = random.choice(sizes)
            u = random.choice([0,1])
            x = random.choice(xshift)
            y = random.choice(yshift)
            gen2(savepath=path, text=w, index=n, fontname=f, size=s, xshift=x, yshift=y, upper=u)

    return 'done'


In [ ]:
# Word stimuli of different length
def CreateMiniWordSet(path_out='wordsets_1000cat_8ex/',num_train=100):
    from scipy.io import loadmat
    
    words = np.load('words4yo.npy')
    wordlist = words[np.linspace(1,len(words)-1,1000,dtype='int')]
    
   
    sizes = [40, 80]
    fonts = ['arial']
    xshift = [-30, 30]
    yshift = [-15, 15]
    
    #create train and val folders 
    for m in ['train']:
        for f in wordlist:
            target_path = path_out+m+'/'+f
            os.makedirs(target_path)
    
    #for each word, create num_train + num_val exemplars, then split randomly into train and val.
    for w in tqdm(wordlist):
        gc.collect()
        print (w,)
        path = path_out+'train/'+w+'/'
        n = 0
        for f in fonts:
            for s in sizes:
                for x in xshift:
                    for y in yshift:
                        n = n+1
                        gen2(savepath=path, text=w, index=n, fontname=f, size=s, xshift=x, yshift=y, upper=1)

    return 'done'


In [ ]:
# Example stimuli to study bigram vs letter model
def CreateExpWords(path_out='stimuli/bigramtest/',num_train=100):
    from scipy.io import loadmat
    
    wordlist = ['ER','RE','EX','XE','RX','XR','XX', 
                'XXXXXERX','XXXXXREX','XXXXXEXX','XXXXXXEX','XXXXXRXX','XXXXXXRX','XXXXXXXX']
    
   
    sizes = [40, 80]
    fonts = ['arial']
    xshift = [-30, 30]
    yshift = [-15, 15]
    
    #create train and val folders 
    for m in ['train']:
        for f in wordlist:
            target_path = path_out+m+'/'+f
            os.makedirs(target_path)
    
    #for each word, create num_train + num_val exemplars, then split randomly into train and val.
    for w in tqdm(wordlist):
        gc.collect()
        print (w,)
        path = path_out+'train/'+w+'/'
        n = 0
        for f in fonts:
            for s in sizes:
                for x in xshift:
                    for y in yshift:
                        n = n+1
                        gen2(savepath=path, text=w, index=n, fontname=f, size=s, xshift=x, yshift=y, upper=1)

    return 'done'

CreateExpWords()

In [ ]:
# Bilingual stimuli
def CreateMiniWordSet(path_out='stimuli/wordsets_bilingual/',num_train=100):
    from scipy.io import loadmat
    
    annots = loadmat('../others/EngFrStim.mat')
    wordlist = np.concatenate(annots['stim'][0])

    sizes = [40, 80]
    fonts = ['arial']
    xshift = [-30, 30]
    yshift = [-15, 15]
    
    #create train and val folders 
    n = 0
    for m in ['train']:
        for f in wordlist:
            target_path = path_out+m+'/'+str(n).zfill(3)+'_'+f
            os.makedirs(target_path)
            n = n+1

    
    #for each word, create num_train + num_val exemplars, then split randomly into train and val.
    n = 0
    for w in tqdm(wordlist):
        gc.collect()
        print (w,)
        path = path_out+'train/'+str(n).zfill(3)+'_'+w+'/'
        n = n+1
#         for f in fonts:
#             for s in sizes:
#                 for x in xshift:
#                     for y in yshift:
#                         n = n+1
#                         gen2(savepath=path, text=w, index=n, fontname=f, size=s, xshift=x, yshift=y, upper=1)

        gen2(savepath=path, text=w, index=1, fontname=fonts[0], size=40, xshift=0, yshift=0, upper=1)

    return 'done'


In [ ]:
# Load words from text file - Vinckier stimuli
import glob, os
import numpy as np
os.walk("../../StimulusProperties/")
wordlist = []; nstimpercat = 50

for file in glob.glob("../../StimulusProperties/*.txt"):
    if not file[-6:-4] == 'NV':
        file1 = open(file,"r") 
        x = (file1.read()).split('\n')
        wordlist = np.concatenate((wordlist, x[:nstimpercat]))

size=60; xshift=0; yshift=0; W = 500; H = 500
for i in range(len(wordlist)):
    img = Image.new("RGB", (W,H), color = (255, 255, 255))
    if i < nstimpercat:
        fontname = 'ff6'
    else:
        fontname = 'deffvmonospaced3'
    text = wordlist[i]    
    fnt = ImageFont.truetype(fontname+'.ttf', size)
    draw = ImageDraw.Draw(img)
    w, h = fnt.getsize(text)
    draw.text((xshift + (W-w)/2, yshift + (H-h)/2), text, font=fnt, fill='black')

    img.save('stimuli/vinckier50/train/allstim/'+str(i).zfill(4) +'_'+text+'.jpg')


In [ ]:
# Jumbled word list
# Example stimuli to study bigram vs letter model
def CreateExpWords(path_out='stimuli/jumbledWords/',num_train=100):
    
    words = np.load('sorted_wordlist.npy')
    wordlen = [len(x) for x in words]
    word6 = words[np.where(np.array(wordlen) ==6)]

    wordlist = word6
    # Letter transposition
    for pos in range(5):
        for i in word6:
            temp = list(i);
            temp[pos] = i[pos+1]; temp[pos+1] = i[pos]
            wordlist = np.append(wordlist,''.join(temp))  
        
    # Letter substitution
    import string
    st = string.ascii_letters
    for pos in range(5):
        for i in word6:
            temp = list(i);
            temp[pos] =  st[np.random.randint(26)] ; temp[pos+1] =  st[np.random.randint(26)]
            wordlist = np.append(wordlist,''.join(temp))  
            
    
    sizes = [60]
    fonts = ['arial']
    xshift = [0]
    yshift = [0]
    
    #create train and val folders 
    n = 0
    for m in ['train']:
        for f in wordlist:
            target_path = path_out+m+'/'+str(n).zfill(4)+'_'+f
            os.makedirs(target_path)
            n = n+1
    
    #for each word, create num_train + num_val exemplars, then split randomly into train and val.
    n = 0
    for w in tqdm(wordlist):
        gc.collect()
        print (w,)
        path = path_out+'train/'+str(n).zfill(4)+'_'+w+'/'
#         for f in fonts:
#             for s in sizes:
#                 for x in xshift:
#                     for y in yshift:
        gen2(savepath=path, text=w, index=n, fontname=fonts[0], size=40, xshift=0, yshift=0, upper=1)
        n = n+1


    return 'done'

CreateExpWords()

In [ ]:
# Rotated word list
def CreateExpWords(path_out='stimuli/rotationstimuli/',num_train=100):
    
    words = np.load('sorted_wordlist.npy')
    wordlen = [len(x) for x in words]
    word6 = words[np.where(np.array(wordlen) ==6)]

    wordlist = word6      
           
    
    sizes = [60]
    fonts = ['arial']
    xshift = [0]
    yshift = [0]
    
    rotation = np.arange(0,100,10)
    #create train and val folders 
    n = 0
    for m in ['train']:
        for f in wordlist:
            target_path = path_out+m+'/'+str(n).zfill(4)+'_'+f
            os.makedirs(target_path)
            n = n+1
    
    #for each word, create num_train + num_val exemplars, then split randomly into train and val.
    n = 0
    for w in tqdm(wordlist):
        indx = 0
        for r in rotation:
            gc.collect()
            print (w,r)
            path = path_out+'train/'+str(n).zfill(4)+'_'+w+'/'
            genrotation(savepath=path, text=w,rotation=r, index=indx, fontname=fonts[0], size=40, xshift=0, yshift=0, upper=1)
            indx = indx + 1
        n = n+1


    return 'done'

CreateExpWords()

In [9]:
# Different spacing word list
def CreateExpWords(path_out='stimuli/spacingstimuli/',num_train=100):
    
    words = np.load('sorted_wordlist.npy')
    wordlen = [len(x) for x in words]
    word6 = words[np.where(np.array(wordlen) ==6)]

    wordlist = word6
    
    fonts = ['arial']
    
    spacing = np.arange(0,2.5,.25)
#     spacing = [0,1]
    # create train and val folders 
    n = 0
    for m in ['train']:
        for f in wordlist:
            target_path = path_out+m+'/'+str(n).zfill(4)+'_'+f
            os.makedirs(target_path)
            n = n+1
    
    #for each word, create num_train + num_val exemplars, then split randomly into train and val.
    n = 0
    for w in tqdm(wordlist):
        indx = 0
        for s in spacing:
            gc.collect()
            print (w,s)
            path = path_out+'train/'+str(n).zfill(4)+'_'+w+'/'
            genspacing(savepath=path, text=w,spacing=s, index=indx, fontname=fonts[0], size=40, upper=1)
            indx = indx + 1
        n = n+1


    return 'done'

CreateExpWords()

  0%|                                                                                          | 0/248 [00:00<?, ?it/s]

adroit 0.0
adroit 0.25
adroit 0.5
adroit 0.75
adroit 1.0
adroit 1.25


  0%|▎                                                                                 | 1/248 [00:00<01:26,  2.86it/s]

adroit 1.5
adroit 1.75
adroit 2.0
adroit 2.25
adulte 0.0
adulte 0.25
adulte 0.5
adulte 0.75
adulte 1.0
adulte 1.25
adulte 1.5
adulte 1.75
adulte 2.0


  1%|▋                                                                                 | 2/248 [00:00<01:26,  2.86it/s]

...

yaourt 0.0
yaourt 0.25
yaourt 0.5
yaourt 0.75
yaourt 1.0
yaourt 1.25
yaourt 1.5


100%|███████████████████████████████████████████████████████████████████████████████▋| 247/248 [01:27<00:00,  2.78it/s]

yaourt 1.75
yaourt 2.0
yaourt 2.25
zigzag 0.0
zigzag 0.25
zigzag 0.5
zigzag 0.75
zigzag 1.0
zigzag 1.25
zigzag 1.5
zigzag 1.75
zigzag 2.0
zigzag 2.25


100%|████████████████████████████████████████████████████████████████████████████████| 248/248 [01:27<00:00,  2.84it/s]


'done'

In [6]:
# Word list for classification
def CreateExpWords(path_out='stimuli/classification_dataset/'):
    
    words = np.load('sorted_wordlist.npy')
    wordlen = [len(x) for x in words]
    word6 = words[np.where(np.array(wordlen) ==6)]

    wordlist = word6
    
    # Middle Letter transposition
    for i in word6:
        temp = list(i); pos = 2
        temp[pos] = i[pos+1]; temp[pos+1] = i[pos]
        wordlist = np.append(wordlist,''.join(temp))  
        
    # Edge Letter transposition
    for i in word6:
        temp = list(i); 
        temp[0] = i[5]; temp[5] = i[0]
        wordlist = np.append(wordlist,''.join(temp)) 
        
    # Middle Letter substitution
    import string
    st = string.ascii_letters
    for i in word6:
        temp = list(i); pos = 2
        temp[pos] =  st[np.random.randint(26)] ; temp[pos+1] =  st[np.random.randint(26)]
        wordlist = np.append(wordlist,''.join(temp))  
    
    # Edge Letter substitution
    import string
    st = string.ascii_letters
    for i in word6:
        temp = list(i); 
        temp[0] =  st[np.random.randint(26)] ; temp[5] =  st[np.random.randint(26)]
        wordlist = np.append(wordlist,''.join(temp))  
    
    
    fonts = ['arial','tahoma']
    #create train and val folders 
    n = 0
    for m in ['train']:
        for f in wordlist:
            target_path = path_out+m+'/'+str(n).zfill(4)+'_'+f
            os.makedirs(target_path)
            n = n+1
    
    #for each word, create num_train + num_val exemplars, then split randomly into train and val.
    n = 0
    for ix, w in enumerate(wordlist):
        gc.collect()
        print (w,)
        path = path_out+'train/'+str(n).zfill(4)+'_'+w+'/'
        if ix < len(word6):
            gen2(savepath=path, text=w, index=n, fontname=fonts[0], size=40, xshift=0, yshift=0, upper=1)
            gen2(savepath=path, text=w, index=n+1, fontname=fonts[1], size=40, xshift=0, yshift=0, upper=1) # Font
            gen2(savepath=path, text=w, index=n+2, fontname=fonts[0], size=70, xshift=0, yshift=0, upper=1) # Size
            gen2(savepath=path, text=w, index=n+3, fontname=fonts[0], size=40, xshift=40, yshift=-20, upper=1) # Location
            gen2(savepath=path, text=w, index=n+4, fontname=fonts[0], size=40, xshift=0, yshift=0, upper=0)  # case
        else:
            gen2(savepath=path, text=w, index=n, fontname=fonts[0], size=40, xshift=0, yshift=0, upper=1)
        n = n+1


    return 'done'

CreateExpWords()

adroit
adulte
agiter
agneau
amener
ananas
animal
anorak
argent
autant
autour
avaler
avance
bagage
baiser
banane
barrer
bassin
bateau
battre
bavoir
bercer
beurre
billet
bonbon
bondir
bonnet
bosser
bottes
bouche
bouger
bouton
brosse
bureau
cabane
cacher
cadeau
cahier
caisse
camion
camper
canard
carnet
carton
casier
casque
casser
cerise
chaise
chemin
cheval
cirque
citron
classe
cloche
cochon
coffre
cogner
coller
contre
copain
copier
coquet
coquin
coudre
couper
courir
course
cousin
couver
crayon
cuisse
danger
danser
debout
dedans
dehors
demain
dessin
devant
dindon
doigts
domino
donner
dormir
douche
droite
effort
encore
endive
enfant
entier
entrer
facile
farine
fermer
feutre
flaque
flocon
fourmi
fraise
frange
gagner
garage
gauche
gentil
goutte
graine
griffe
hanche
humide
hurler
imiter
intrus
jaloux
jamais
jambes
jambon
jardin
joyeux
jumeau
laisse
lancer
langue
lavabo
lettre
louche
madame
maigre
maison
mamans
manche
manger
marche
marier
marron
masque
mettre
meuble
milieu
minuit
minute
monter

ioulet
xrisej
driver
vroprf
huzzlb
syjams
juatro
daisib
uampeh
eangec
tapidr
fasoih
yayuri
eegarp
gemuef
uenarp
pendrg
zetarz
oetoux
mevoiu
zidear
yongej
coules
waisob
ialadz
malueu
zamedp
rautel
mauvex
qavoit
lentiz
nerrej
nervif
biesto
voldak
zoleib
uombre
mommet
oonneh
eortia
bortip
zouplo
kquare
xuivrp
baillh
vampos
nendrr
eickeu
kimbrz
riroiy
romats
jombej
dortul
cristk
lrouep
zulipw
eunner
salism
iented
xentrw
tersex
liandu
biragw
zoisig
moyago
caourp
nigzau


'done'

In [ ]:
# CreateMiniWordSet()